# Decentralized Multi-Agent Pathfinding: A Game-Theoretic Approach

### Introduction

In contrast to **centralized** planners which possess a global "god's-eye view" to compute optimal plans for all agents simultaneously, **decentralized** approaches grant autonomy to individual agents. Each agent plans for itself and coordinates with others to resolve conflicts.

We can frame this coordination challenge as a cooperative game. The algorithm we will model is **Prioritized Planning**, where agents follow a strict social hierarchy to negotiate who gets the right-of-way.

***

### The Game: Prioritized Planning

This is a **fully cooperative game** where all players win or lose together.

* **Players**: A set of $k$ agents, $\mathcal{A} = \{A_1, A_2, \dots, A_k\}$.
* **Objective**: To find a set of conflict-free paths, $\Pi = \{\pi_1, \pi_2, \dots, \pi_k\}$, that brings every agent from its start to its goal. The team's score is the **sum-of-costs**, which we want to minimize.
    \begin{aligned}
    \text{Cost}(\Pi) = \sum_{i=1}^{k} |\pi_i|
    \end{aligned}
    where $|\pi_i|$ is the length (cost) of the path for agent $A_i$.

* **Agent Capability**: Each agent $A_i$ is equipped with a single-agent pathfinding algorithm (e.g., A*) capable of finding an optimal path for itself given a set of space-time constraints.

***

### The Rules of Play (The Algorithm)

The game proceeds sequentially according to a pre-defined "social rule" of priority.

#### Step 1: Establish Priority
A strict, total ordering is defined over the agents before planning begins. This priority ranking is fixed.
\begin{aligned}
A_1 > A_2 > \dots > A_k
\end{aligned}

#### Step 2: Sequential Pathfinding
The agents plan one-by-one, from highest to lowest priority. The paths of higher-priority agents become obstacles for all subsequent agents.

1.  **Highest-Priority Agent ($A_1$)**: Agent $A_1$ plans its path $\pi_1$ in a completely empty environment. It solves for its own true shortest path. The initial set of constraints $\mathcal{C}_0$ is empty.
    \begin{aligned}
    \pi_1 = \arg\min_{\pi} \text{cost}(\pi \mid \mathcal{C}_0 = \emptyset)
    \end{aligned}

2.  **Subsequent Agents ($A_i$)**: Every subsequent agent $A_i$ must compute its shortest path while respecting the paths of all previously planned agents $\{A_1, \dots, A_{i-1}\}$.
    Let $\mathcal{C}_{i-1}$ be the set of all space-time constraints imposed by the paths of agents $A_1$ through $A_{i-1}$. Agent $A_i$ solves:
    \begin{aligned}
    \pi_i = \arg\min_{\pi} \text{cost}(\pi \mid \mathcal{C}_{i-1})
    \end{aligned}

3.  **Updating Constraints**: After finding its path $\pi_i$, the locations occupied by agent $A_i$ at each timestep are added to the constraint set for all agents that follow. The new constraint set $\mathcal{C}_i$ becomes:
    \begin{aligned}
    \mathcal{C}_i = \mathcal{C}_{i-1} \cup \{ (\pi_i(t), t) \mid t \in [0, \infty) \}
    \end{aligned}
    Note: The constraint $(\text{loc}, t)$ means that the location `loc` is occupied at timestep `t`. We assume the agent occupies its goal location for all time after arriving to prevent future collisions.

This process continues until the lowest-priority agent has successfully planned its path.

***

### Analysis and Trade-offs

This decentralized "game" is a powerful heuristic, but it comes with significant trade-offs compared to a centralized approach.

* **Optimality**: Prioritized Planning is **not optimal**. A high-priority agent might choose a path that is best for itself but forces a low-priority agent to take a massive, inefficient detour. A globally optimal planner would have found a better compromise where a small sacrifice by the high-priority agent leads to a much better overall solution cost.

* **Completeness**: The algorithm is **not complete**. It is possible for a high-priority agent's path to completely block any possible route for a low-priority agent, causing the algorithm to fail to find a solution even when one exists.

* **Complexity**: The primary advantage is speed. The time complexity is polynomial in the number of agents, roughly $k \times \text{Cost(A*)}$. This allows it to solve problems with many more agents than a centralized planner could ever handle. It trades guarantees of optimality and completeness for scalability and performance.

==================================================
    BENCHMARK RESULTS (Decentralized Prioritized)
==================================================
Solution Found!
  - Solution Cost (Total Timesteps): 38
  - Execution Time: 0.0044 seconds
  - Peak Memory Usage: 0.074 MB
  - States/Nodes Processed: 40